In [1]:
import numpy as np
import pandas as pd
import sys

import config as cfg
from functools import reduce
import joblib,time
from tkinter import _flatten
sys.path.append("./tools/")
import funclib,os
import exact_ec_from_uniprot as exactec
import minitools as mtool
import benchmark_common as bcommon
import embedding_esm as esmebd
from tools.Attention import Attention
from keras.models import load_model
from pandarallel import pandarallel 
pandarallel.initialize() 
import benchmark_train as btrain

%load_ext autoreload
%autoreload 2

2023-02-24 05:07:06.195106: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 05:07:06.283373: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
input_file = cfg.DATADIR + 'sample_25.fasta'
output_tsv=output_file = cfg.RESULTSDIR + 'ec_res20230206.tsv'
topk = 20
compute_mode = 'r'

In [4]:
start = time.process_time()
# 1. 读入数据
print('step 1: loading data')
input_df = funclib.load_fasta_to_table(input_file) # test fasta
latest_sprot = pd.read_feather(cfg.FILE_LATEST_SPROT_FEATHER) #sprot db

# 2. 查找数据
print('step 2: find existing data')
find_data =input_df.merge(latest_sprot, on='seq', how='left')
find_data=latest_sprot[latest_sprot.seq.isin(input_df.seq)]
exist_data = find_data.merge(input_df, on='seq', how='left').iloc[:,np.r_[8,0,1:8]].rename(columns={'id_x':'uniprot_id','id_y':'input_id'}).reset_index(drop=True)
noExist_data = input_df[~input_df.seq.isin(find_data.seq)]

if len(noExist_data) == 0:
    exist_data=exist_data[['input_id','ec_number']].rename(columns={'ec_number':'ec_pred'})
    exist_data.to_csv(output_tsv, sep='\t', index=False)
    end = time.process_time()
    print('All done running time: %s Seconds'%(end-start))

step 1: loading data
step 2: find existing data


In [8]:
rep0, rep32, rep33 = esmebd.get_rep_multi_sequence(sequences=input_df, model='esm1b_t33_650M_UR50S',seqthres=1022)
# 4. sequence alignment
print('step 4: sequence alignment')
if not os.path.exists(cfg.FILE_BLAST_PRODUCTION_DB):
    funclib.table2fasta(latest_sprot, cfg.FILE_BLAST_PRODUCTION_FASTA)
    cmd = r'diamond makedb --in {0} -d {1}'.format(cfg.FILE_BLAST_PRODUCTION_FASTA, cfg.FILE_BLAST_PRODUCTION_DB)
    os.system(cmd)

blast_res = funclib.getblast_usedb(db=cfg.FILE_BLAST_PRODUCTION_DB, test=input_df)

blast_res =blast_res[['id', 'sseqid']].merge(latest_sprot, left_on='sseqid', right_on='id', how='left').iloc[:,np.r_[0,2,3:10]].rename(columns={'id_x':'input_id','id_y':'uniprot_id'}).reset_index(drop=True)

100%|██████████| 31/31 [00:27<00:00,  1.13it/s]
diamond v0.9.30.131 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org



step 4: sequence alignment
Write finished
diamond blastp -d /home/shizhenkun/codebase/DMLF/data/uniprot_blast_db/production_blast.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet


In [11]:
label_model_ec = pd.read_feather(f'{cfg.MODELDIR}/task3_labels.feather').label_multi.to_list()
model_ec = load_model('/home/shizhenkun/codebase/DMLF/model/task3_esm32_2022.h5',custom_objects={"Attention": Attention}, compile=False)
predicted = model_ec.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
sorted(dict(zip((label_model_ec), predicted[30])).items(),key = lambda x:x[1], reverse = True)[0:10]

1/1 [==============================] - 0s 396ms/step


[('2.6.1.1', 0.8077546),
 ('2.6.1.57', 0.535273),
 ('2.6.1.5', 0.16525973),
 ('2.6.1.107', 0.054249667),
 ('2.6.1.83', 0.0126633225),
 ('4.1.1.81', 0.008570874),
 ('2.6.1.3', 0.008404819),
 ('4.4.1.8', 0.007159066),
 ('2.6.1.7', 0.0017221959),
 ('2.6.1.-', 0.0011435368)]

In [36]:
output_df=pd.DataFrame()
output_df['id']=input_df['id'].copy()
output_df['recomendations']=pd.DataFrame(predicted).apply(lambda x :sorted(dict(zip((label_model_ec), x)).items(),key = lambda x:x[1], reverse = True)[0:10], axis=1 ).values

In [39]:
output_df

,id,recomendations
0,sp|O00625|PIR_HUMAN,"[(1.13.11.24, 0.949963390827179), (1.16.3.1, 0..."
1,sp|P06028|GLPB_HUMAN,"[(3.4.-.-, 0.07833539694547653), (2.4.99.18, 0..."
2,sp|P06746|DPOLB_HUMAN,"[(4.2.99.-, 0.9834648370742798), (2.7.7.7, 0.9..."
3,sp|P09588|H2A_CAEEL,"[(2.7.7.6, 0.003033645683899522), (2.7.7.7, 0...."
4,sp|P0A7M9|RL31_ECOLI,"[(3.1.-.-, 0.012678814120590687), (1.5.3.1, 0...."
5,sp|P0DJD7|PEPA4_HUMAN,"[(3.4.23.1, 0.9049341082572937), (3.4.23.34, 0..."
6,sp|P24941|CDK2_HUMAN,"[(2.7.11.22, 0.948376476764679), (2.7.11.23, 0..."
7,sp|P30843|BASR_ECOLI,"[(2.7.13.3, 0.31582269072532654), (3.6.3.-, 0...."
8,sp|P49842|STK19_HUMAN,"[(2.7.11.1, 0.9927850961685181), (3.1.21.4, 0...."
9,sp|P58107|EPIPL_HUMAN,"[(2.3.2.-, 0.019711995497345924), (2.7.4.6, 0...."


In [27]:
pd.DataFrame(predicted).apply(lambda x :sorted(dict(zip((label_model_ec), x)).items(),key = lambda x:x[1], reverse = True)[0:10], axis=1 ).values

array([list([('1.13.11.24', 0.949963390827179), ('1.16.3.1', 0.006510687991976738), ('1.13.11.20', 0.00333215924911201), ('1.13.11.-', 0.002859009662643075), ('1.2.3.4', 0.002591111231595278), ('5.-.-.-', 0.00206043035723269), ('5.1.3.13', 0.0018545443890616298), ('4.1.1.2', 0.0016180051025003195), ('4.4.1.5', 0.0014525667065754533), ('3.4.13.18', 0.0012993296841159463)]),
       list([('3.4.-.-', 0.07833539694547653), ('2.4.99.18', 0.008385669440031052), ('2.3.2.27', 0.007210354320704937), ('1.6.5.-', 0.0045473333448171616), ('3.-.-.-', 0.0030648880638182163), ('3.1.1.-', 0.0029117504600435495), ('3.2.1.18', 0.0021186743397265673), ('3.1.1.59', 0.002095003379508853), ('2.3.1.20', 0.0018828455358743668), ('2.4.1.198', 0.0017801284557208419)]),
       list([('4.2.99.-', 0.9834648370742798), ('2.7.7.7', 0.9631432890892029), ('4.2.99.18', 0.029926210641860962), ('2.7.7.31', 0.017365694046020508), ('3.6.4.12', 0.00445278687402606), ('6.5.1.2', 0.003579071257263422), ('3.2.2.-', 0.001044134

In [29]:
output_df

0                                               sp|O00625|PIR_HUMAN
1                                              sp|P06028|GLPB_HUMAN
2                                             sp|P06746|DPOLB_HUMAN
3                                               sp|P09588|H2A_CAEEL
4                                              sp|P0A7M9|RL31_ECOLI
5                                             sp|P0DJD7|PEPA4_HUMAN
6                                              sp|P24941|CDK2_HUMAN
7                                              sp|P30843|BASR_ECOLI
8                                             sp|P49842|STK19_HUMAN
9                                             sp|P58107|EPIPL_HUMAN
10                                              sp|P68431|H31_HUMAN
11                                            sp|P78552|I13R1_HUMAN
12                                             sp|P90994|DJ11_CAEEL
13                                             sp|Q12816|TROP_HUMAN
14                                            sp

In [5]:
input_df

,id,seq
0,sp|O00625|PIR_HUMAN,MGSSKKVTLSVLSREQSEGVGARVRRSIGRPELKNLDPFLLFDEFK...
1,sp|P06028|GLPB_HUMAN,MYGKIIFVLLLSEIVSISALSTTEVAMHTSTSSSVTKSYISSQTNG...
2,sp|P06746|DPOLB_HUMAN,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...
3,sp|P09588|H2A_CAEEL,MSGRGKGGKAKTGGKAKSRSSRAGLQFPVGRLHRILRKGNYAQRVG...
4,sp|P0A7M9|RL31_ECOLI,MKKDIHPKYEEITASCSCGNVMKIRSTVGHDLNLDVCSKCHPFFTG...
5,sp|P0DJD7|PEPA4_HUMAN,MKWLLLLGLVALSECIMYKVPLIRKKSLRRTLSERGLLKDFLKKHN...
6,sp|P24941|CDK2_HUMAN,MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPS...
7,sp|P30843|BASR_ECOLI,MKILIVEDDTLLLQGLILAAQTEGYACDSVTTARMAEQSLEAGHYS...
8,sp|P49842|STK19_HUMAN,MQKWFSAFDDAIIQRQWRANPSRGGGGVSFTKEVDTNVATGAPPRR...
9,sp|P58107|EPIPL_HUMAN,MSGHTLPPLPVPGTNSTEQASVPRAMAATLGAGTPPRPQARSIAGV...


In [66]:
predicted[30]

array([1.0128557e-14, 4.1971328e-20, 2.4246384e-16, ..., 3.1200917e-20,
       4.2411721e-18, 2.9364197e-17], dtype=float32)

In [36]:
predicted1=predicted.copy()

In [38]:
predicted1=1
predicted1

1

In [49]:
dir(encoder_t3)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_cache',
 '_cached_dict',
 '_check_feature_names',
 '_check_n_features',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_sklearn_auto_wrap_output_keys',
 '_transform',
 '_validate_data',
 '_validate_params',
 'classes',
 'classes_',
 'fit',
 'fit_transform',
 'get_params',
 'inverse_transform',
 'set_output',
 'set_params',
 'sparse_output',
 'transform']

In [42]:
[[1 for item in predicted[0]]]

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [8]:
pred_dmlf

,id,dmlf_isEnzyme,dmlf_howmany,dmlf_ec
0,sp|O00625|PIR_HUMAN,True,1,1.13.11.24
1,sp|P06028|GLPB_HUMAN,False,1,2.4.1.198
2,sp|P06746|DPOLB_HUMAN,True,2,2.7.7.7
3,sp|P09588|H2A_CAEEL,False,1,4.2.99.18
4,sp|P0A7M9|RL31_ECOLI,False,1,1.8.4.11
5,sp|P0DJD7|PEPA4_HUMAN,True,1,3.4.23.1
6,sp|P24941|CDK2_HUMAN,True,1,2.7.11.22
7,sp|P30843|BASR_ECOLI,False,1,2.7.13.3
8,sp|P49842|STK19_HUMAN,True,1,3.1.22.-
9,sp|P58107|EPIPL_HUMAN,False,1,4.1.99.12


In [22]:


print('step 8: integrate results')

# output_df = integrate_out_put(existing_table=exist_data,
#                               blast_table=blast_res,
#                               dmlf_pred_table = pred_dmlf,
#                               mode=mode,
#                               topnum=topnum
#                             )  
results = pred_dmlf.merge(exist_data, left_on='id',right_on='input_id', how='left') 
results=results.fillna('#')
results['ec_pred'] =results.apply(lambda x : x.ec_number if x.ec_number!='#' else ('-' if x.dmlf_isEnzyme==False else x.dmlf_ec) ,axis=1)
output_df = results[['id', 'ec_pred']].rename(columns={'id':'id_input'})

print('step 9: writting results') 

output_df.to_csv(output_tsv, sep='\t', index=False)

end = time.process_time()
print('All done running time: %s Seconds'%(end-start))

In [21]:
latest_sprot.to_feather(cfg.FILE_LATEST_SPROT_FEATHER)

In [36]:
esm32=pd.read_feather(cfg.FILE_FEATURE_ESM32)

In [38]:
esm32[esm32.id=='P04693']

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f1271,f1272,f1273,f1274,f1275,f1276,f1277,f1278,f1279,f1280
326082,P04693,1.463843,1.112741,-2.322964,0.096209,-5.257494,-1.595543,-1.007157,-2.500327,-3.963384,...,12.347787,1.569019,-3.010229,3.729503,-5.833245,-8.200345,4.29761,-5.090612,-5.80144,0.801188


In [28]:
latest_sprot[latest_sprot.id=='P04693'].seq.values[0]

'MFQKVDAYAGDPILTLMERFKEDPRSDKVNLSIGLYYNEDGIIPQLQAVAEAEARLNAQPHGASLYLPMEGLNCYRHAIAPLLFGADHPVLKQQRVATIQTLGGSGALKVGADFLKRYFPESGVWVSDPTWENHVAIFAGAGFEVSTYPWYDEATNGVRFNDLLATLKTLPARSIVLLHPCCHNPTGADLTNDQWDAVIEILKARELIPFLDIAYQGFGAGMEEDAYAIRAIASAGLPALVSNSFSKIFSLYGERVGGLSVMCEDAEAAGRVLGQLKATVRRNYSSPPNFGAQVVAAVLNDEALKASWLAEVEEMRTRILAMRQELVKVLSTEMPERNFDYLLNQRGMFSYTGLSAAQVDRLREEFGVYLIASGRMCVAGLNTANVQRVAKAFAAVM'

In [8]:
exist_data=exist_data[['input_id','ec_number']].rename(columns={'ec_number':'ec_pred'})
exist_data


,input_id,ec_pred
0,sp|P0A7M9|RL31_ECOLI,-
1,sp|P68431|H31_HUMAN,-
2,sp|P06028|GLPB_HUMAN,-
3,sp|P06746|DPOLB_HUMAN,"4.2.99.-,2.7.7.7"
4,sp|P09588|H2A_CAEEL,-
5,sp|P24941|CDK2_HUMAN,2.7.11.22
6,sp|P30843|BASR_ECOLI,-
7,sp|P49842|STK19_HUMAN,2.7.11.1
8,sp|P53017|MSP19_CAEEL,-
9,sp|P78552|I13R1_HUMAN,-


In [4]:
# 3. EMBedding
mode='r'
print('step 3: Embedding')
if mode =='p':
    rep0, rep32, rep33 = esmebd.get_rep_multi_sequence(sequences=noExist_data, model='esm1b_t33_650M_UR50S',seqthres=1022)
if mode == 'r':
    rep0, rep32, rep33 = esmebd.get_rep_multi_sequence(sequences=input_df, model='esm1b_t33_650M_UR50S',seqthres=1022)

step 3: Embedding


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:40<00:00,  1.36s/it]


In [5]:
# 4. sequence alignment
print('step 4: sequence alignment')
if not os.path.exists(cfg.FILE_BLAST_PRODUCTION_DB):
    funclib.table2fasta(latest_sprot, cfg.FILE_BLAST_PRODUCTION_FASTA)
    cmd = r'diamond makedb --in {0} -d {1}'.format(cfg.FILE_BLAST_PRODUCTION_FASTA, cfg.FILE_BLAST_PRODUCTION_DB)
    os.system(cmd)

step 4: sequence alignment


In [6]:
if mode =='p':
    blast_res = funclib.getblast_usedb(db=cfg.FILE_BLAST_PRODUCTION_DB, test=noExist_data)
if mode == 'r':
    blast_res = funclib.getblast_usedb(db=cfg.FILE_BLAST_PRODUCTION_DB, test=input_df)
    
    
blast_res =blast_res[['id', 'sseqid']].merge(latest_sprot, left_on='sseqid', right_on='id', how='left').iloc[:,np.r_[0,2,3:10]].rename(columns={'id_x':'input_id','id_y':'uniprot_id'}).reset_index(drop=True)


Write finished
diamond blastp -d /home/shizhenkun/codebase/DMLF/data/uniprot_blast_db/production_blast.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet


In [8]:
pred_dmlf = pd.DataFrame(rep32.id.copy())
model_isEnzyme = load_model(cfg.ISENZYME_MODEL,custom_objects={"Attention": Attention}, compile=False)
predicted = model_isEnzyme.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
encoder_t1=joblib.load(cfg.DICT_LABEL_T1)
pred_dmlf['dmlf_isEnzyme']=(encoder_t1.inverse_transform(bcommon.props_to_onehot(predicted))).reshape(1,-1)[0]

1/1 [==============================] - 0s 391ms/step


In [9]:
pred_dmlf

,id,dmlf_isEnzyme
0,sp|O00625|PIR_HUMAN,False
1,sp|P06028|GLPB_HUMAN,False
2,sp|P06746|DPOLB_HUMAN,False
3,sp|P09588|H2A_CAEEL,False
4,sp|P0A7M9|RL31_ECOLI,False
5,sp|P0DJD7|PEPA4_HUMAN,False
6,sp|P24941|CDK2_HUMAN,False
7,sp|P30843|BASR_ECOLI,False
8,sp|P49842|STK19_HUMAN,False
9,sp|P58107|EPIPL_HUMAN,False


In [10]:
model_howmany = load_model(cfg.HOWMANY_MODEL,custom_objects={"Attention": Attention}, compile=False)
predicted = model_howmany.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
encoder_t2=joblib.load(cfg.DICT_LABEL_T2)
pred_dmlf['dmlf_howmany']=(encoder_t2.inverse_transform(bcommon.props_to_onehot(predicted))).reshape(1,-1)[0]

1/1 [==============================] - 0s 386ms/step


In [11]:
model_ec = load_model(cfg.EC_MODEL,custom_objects={"Attention": Attention}, compile=False)
predicted = model_ec.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
encoder_t3=joblib.load(cfg.DICT_LABEL_T3)
pred_dmlf['dmlf_ec']=[','.join(item) for item in (encoder_t3.inverse_transform(bcommon.props_to_onehot(predicted)))]

1/1 [==============================] - 0s 380ms/step


In [12]:
pred_dmlf

,id,dmlf_isEnzyme,dmlf_howmany,dmlf_ec
0,sp|O00625|PIR_HUMAN,False,1,1.1.2.7
1,sp|P06028|GLPB_HUMAN,False,1,3.1.1.1
2,sp|P06746|DPOLB_HUMAN,False,1,2.4.2.4
3,sp|P09588|H2A_CAEEL,False,5,3.1.13.2
4,sp|P0A7M9|RL31_ECOLI,False,3,3.4.22.-
5,sp|P0DJD7|PEPA4_HUMAN,False,4,1.1.1.35
6,sp|P24941|CDK2_HUMAN,False,1,2.7.7.23
7,sp|P30843|BASR_ECOLI,False,1,3.1.27.3
8,sp|P49842|STK19_HUMAN,False,1,2.4.2.3
9,sp|P58107|EPIPL_HUMAN,False,3,2.4.2.3
